In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Data Preprocessing

In [2]:
df = pd.read_csv('AB_test_data.csv')

In [3]:
df = df.rename(columns={'purchase_TF': 'conversion', 'Variant': 'variant'})
df.head()

,conversion,variant,date,id
0,False,A,2019-11-08,0x25b44a
1,False,B,2020-08-27,0x46271e
2,False,A,2020-06-11,0x80b8f1
3,False,B,2020-08-22,0x8d736d
4,False,A,2020-08-05,0x96c9c8


In [4]:
df["conversion"].value_counts()

False    110415
True      19585
Name: conversion, dtype: int64

In [5]:
df['variant'].value_counts()

A    125000
B      5000
Name: variant, dtype: int64

In [6]:
con_values = {
    False: 0,
    True: 1
}

df.conversion = df.conversion.map(con_values)
df.head()

,conversion,variant,date,id
0,0,A,2019-11-08,0x25b44a
1,0,B,2020-08-27,0x46271e
2,0,A,2020-06-11,0x80b8f1
3,0,B,2020-08-22,0x8d736d
4,0,A,2020-08-05,0x96c9c8


In [7]:
df.conversion.value_counts()

0    110415
1     19585
Name: conversion, dtype: int64

In [8]:
df[df['date']>'2020-08-01']['variant'].value_counts()

B    4832
A    4814
Name: variant, dtype: int64

## Hypothesis Test

**Definition**

"A" -- control group -- shows current practice; 

"B" -- treatment group -- adds walkability assessment (user review)

* Null hypothesis: R_B = R_A
* Alternative hypothesis: R_B > R_A

In [9]:
from scipy.stats import norm

In [10]:
control_results = df[df['variant'] == 'A']['conversion']
treatment_results = df[df['variant'] == 'B']['conversion']

In [11]:
p_hat = treatment_results.sum()/treatment_results.count()
p = control_results.sum()/control_results.count()
sigma_p_hat = np.sqrt((p*(1-p)/treatment_results.count()))

In [12]:
z = (p_hat-p)/sigma_p_hat
z

5.349273094732516

In [13]:
p_value = norm().sf(z) #sf = 1-CDF
p_value

4.415410062469104e-08

**Reject null hypothesis if z>z_a:**

when a = 5%, z_a = 1.64, we reject the null hypothesis and state that there exists enough evidence to state that adding walkability assessment **will improve the conversion of the customers**. 

## Choosing the optimal sample size

In [14]:
#t_(a/2)
norm().ppf(1-0.05/2)

1.959963984540054

In [15]:
#t_(beta)
norm().ppf(1-0.2)

0.8416212335729143

Following the standards in the literature and using a signifcance level of 0.05, and setting power to 0.80, we have
t_(a/2) = 1.96 and t_beta = 0.84 from standard normal tables.

In [16]:
p_bar = (p_hat+p)/2
p_bar

0.163108

In [17]:
mdf = p_hat-p

In [18]:
sample_size=((norm().ppf(1-0.05/2))*np.sqrt(2*p_bar*(1-p_bar)) + \
             (norm().ppf(1-0.2))*np.sqrt(p*(1-p)+p_hat*(1-p_hat))
            )/(mdf**2)
sample_size

2009.978857988649

In [19]:
sample_c = []
sample_t = []
i = 0
while i < 10:
    control = control_results.sample(n=round(sample_size))
    sample_c.append(control)
    treatment = treatment_results.sample(n=round(sample_size))
    sample_t.append(treatment)
    p_hat = treatment.sum()/treatment.count()
    p = control.sum()/control.count()
    sigma_p_hat = np.sqrt((p*(1-p)/treatment.count()))
    z = (p_hat-p)/sigma_p_hat
    p_value = norm().sf(z)
    print("sample %d: , z score: %f, p_value: %f"%(i,z,p_value))
    if z>1.64:
        print('Reject the null hypothesis.')
    else:
        print('Fail to reject the null hypothesis.')
    i+=1

sample 0: , z score: 3.729332, p_value: 0.000096
Reject the null hypothesis.
sample 1: , z score: 1.795729, p_value: 0.036269
Reject the null hypothesis.
sample 2: , z score: 2.509134, p_value: 0.006051
Reject the null hypothesis.
sample 3: , z score: 4.694622, p_value: 0.000001
Reject the null hypothesis.
sample 4: , z score: 3.601496, p_value: 0.000158
Reject the null hypothesis.
sample 5: , z score: 5.395774, p_value: 0.000000
Reject the null hypothesis.
sample 6: , z score: 1.793348, p_value: 0.036459
Reject the null hypothesis.
sample 7: , z score: 3.263934, p_value: 0.000549
Reject the null hypothesis.
sample 8: , z score: 3.695755, p_value: 0.000110
Reject the null hypothesis.
sample 9: , z score: 4.754172, p_value: 0.000001
Reject the null hypothesis.


## Sequential Test

In [20]:
sample_c[0][0:1]

22572    0
Name: conversion, dtype: int64

In [21]:
sample_c[0]

22572     0
103071    0
42141     1
86491     0
108166    0
         ..
29670     1
70188     0
101096    0
92164     0
10141     0
Name: conversion, Length: 2010, dtype: int64

In [22]:
#boundary
upper = np.log(1/0.05) #alpha = 0.05
lower = np.log(0.2) #beta = 0.2

In [23]:
round(sample_size)

2010

In [24]:
trials=[]
i = 0
while i<10:
    ln = 0
    control = sample_c[i]
    treatment = sample_t[i]
    p = control.sum()/control.count()
    p_hat = treatment.sum()/treatment.count()
    j = 0
    while j < round(sample_size):
        d = treatment[j:j+1]
        if int(d) == 1:
            likelihood = np.log(p_hat/p)
        elif int(d) == 0:
            likelihood = np.log((1-p_hat)/(1-p))
        
        ln += likelihood
        
        if ln <= lower:
            trials.append(j+1)
            print('sample: %d, index: %d'%(i, j))
            print(ln)
            print('since ln(lambda_n) is lower than the boundary, we accept Ho.')
            break    
        
        elif ln >= upper:
            trials.append(j+1)
            print('sample: %d, index: %d'%(i, j))
            print(ln)
            print('since ln(lambda_n) is higher than the boundary, we reject Ho.')
            break
            
        elif (lower < ln < upper) and (j == round(sample_size)-1):
            trials.append(j+1)
            print('sample: %d'%(i))
            print(ln)
            print('Cannot stop the test prior to using the full sample.')
            
        j+=1
       
    i+=1
print('The average iteration is: %.0f'%(sum(trials)/len(trials)))

sample: 0, index: 675
3.0104901608472985
since ln(lambda_n) is higher than the boundary, we reject Ho.
sample: 1, index: 1125
3.047275131830145
since ln(lambda_n) is higher than the boundary, we reject Ho.
sample: 2, index: 375
-1.618175637698847
since ln(lambda_n) is lower than the boundary, we accept Ho.
sample: 3, index: 304
3.204206720780097
since ln(lambda_n) is higher than the boundary, we reject Ho.
sample: 4, index: 647
3.0495960218833233
since ln(lambda_n) is higher than the boundary, we reject Ho.
sample: 5, index: 199
3.095946051810562
since ln(lambda_n) is higher than the boundary, we reject Ho.
sample: 6
1.5687855697911757
Cannot stop the test prior to using the full sample.
sample: 7, index: 693
3.1293912524693615
since ln(lambda_n) is higher than the boundary, we reject Ho.
sample: 8, index: 798
3.035758474127336
since ln(lambda_n) is higher than the boundary, we reject Ho.
sample: 9, index: 480
3.1952279218995345
since ln(lambda_n) is higher than the boundary, we reject